In [5]:
from collections import defaultdict
import numpy as np
import sys
from enum import Enum
class ACTION(Enum):
    high=1
    low=0 
    
class Winner:
    def __init__(self):
        self.P_H = 0.55
        self.P_L = 0.45
        self.state = 0
        self.done = 0
        self.R = 0
        self.highcost = -50
        self.lowcost = -10
        self.greedy = 0.1
        self.discount=0.9
        self.init_Q()
        self.alpha=0.1
    def reset_game(self):
        self.state=0
        self.done=0
    def ba_policy(self,state):
        smp = np.random.uniform(0,1)
        if smp >= self.greedy :
            if self.Q[state,ACTION.high] > self.Q[state,ACTION.low]:
                ac = ACTION.high
            else:
                ac = ACTION.low
        else:
            act = np.random.randint(0, 2)
            if act == 0:
                ac = ACTION.low
            else:
                ac = ACTION.high
        return ac
        
    #def ta_policy(self):
    def SARSA(self,numepisode):
        self.init_Q()
        #self.init_C()
        for i_episode in range(1, numepisode+1):
            if i_episode % 1000 == 0:
                print("\rEpisode {}/{}.".format(i_episode, numepisode), end="")
                sys.stdout.flush()
            
            self.reset_game()
            state=0
            action = self.ba_policy(state)
            while(1):
                reward,next_state,done = self.env_step(state,action)
                next_action = self.ba_policy(next_state)
                self.Q[state,action] = self.Q[state,action] + self.alpha*(reward + self.discount*self.Q[next_state,next_action]-self.Q[state,action])
                state = next_state
                action = next_action
                if done == 1:
                    break

           
        return self.Q
        
        
    def env_step(self,state,action):
        done=0
        reward=0
        sample = np.random.uniform(0, 1)
        if action == ACTION.high:
            reward = reward + self.highcost
            suc_prob = self.P_H
        else:
            suc_prob = self.P_L
            reward = reward + self.lowcost
        if sample < suc_prob:
            state = state + 1
        else:
            state = state -1
        if state >= 3 :
            reward = reward + 1000
            done = 1
        if state <= (-3):
            done = 1
        return reward,state,done
    def init_Q(self):
        self.Q = defaultdict(lambda:np.zeros(1))
        for state in range(7):
            for act in ACTION:
                    self.Q[state-3,act]=0.0
    def init_C(self):
        
        self.C = defaultdict(lambda:np.zeros(1))
        for state in range(7):
            for act in ACTION:
                    self.C[state-3,act]=0.0
        

In [6]:
winner=Winner()

Q=winner.SARSA(500000)
print('policy_value',Q)
import pandas as pd
buf = np.zeros((7,2))
for i in range(7):
    for j in ACTION:
        buf[i][j.value]=Q[i-3,j]

test_3 = pd.DataFrame(buf)
print(test_3)

Episode 500000/500000.policy_value defaultdict(<function Winner.init_Q.<locals>.<lambda> at 0x7fb6041d6378>, {(-3, <ACTION.high: 1>): 0.0, (-3, <ACTION.low: 0>): 0.0, (-2, <ACTION.high: 1>): -31.679559585346585, (-2, <ACTION.low: 0>): -1.8888515788383105, (-1, <ACTION.high: 1>): 8.734128693657883, (-1, <ACTION.low: 0>): 52.84632536754146, (0, <ACTION.high: 1>): 87.0753813411271, (0, <ACTION.low: 0>): 150.60913783310164, (1, <ACTION.high: 1>): 356.0957456127873, (1, <ACTION.low: 0>): 238.57220302711343, (2, <ACTION.high: 1>): 685.4575269630155, (2, <ACTION.low: 0>): 438.40222386920897, (3, <ACTION.high: 1>): 0.0, (3, <ACTION.low: 0>): 0.0})
            0           1
0    0.000000    0.000000
1   -1.888852  -31.679560
2   52.846325    8.734129
3  150.609138   87.075381
4  238.572203  356.095746
5  438.402224  685.457527
6    0.000000    0.000000
